<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/12_5_minibatch_stochastic_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00


## 12.5 Minibatch Stochastic Gradient Descent

### 12.5.1 Vectorization and Caches

In [2]:
%matplotlib inline
import time
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
A = torch.zeros(256, 256)
B = torch.randn(256, 256)
C = torch.randn(256, 256)

In [5]:
class Timer:
  def __init__(self):
    self.times = []
    self.start()

  def start(self):
    self.tik = time.time()

  def stop(self):
    self.times.append(time.time() - self.tik)
    return self.times[-1]

  def avg(self):
    return sum(self.times) / len(self.times)

  def sum(self):
    return sum(self.times)

  def cumsum(self):
    return np.array(self.times).cumsum().tolist()

timer = Timer()

In [6]:
# Compute A = BC one element at a time
timer.start()
for i in range(256):
  for j in range(256):
    A[i, j] = torch.dot(B[i, :], C[:, j])
timer.stop()

1.885279893875122

In [7]:
# Compute A = BC one column at a time
timer.start()
for j in range(256):
  A[:, j] = torch.mv(B, C[:, j])
timer.stop()

0.022409915924072266

In [10]:
## COmpute A = BC in one go
timer.start()
A = torch.mm(B, C)
timer.stop()

gigaflops = [0.03 / i for i in timer.times]
print(f'performance in Gigaflops: element {gigaflops[0]: .3f}, '
      f'column {gigaflops[1]: .3f}, full {gigaflops[2]: .3f}')

performance in Gigaflops: element  0.016, column  1.339, full  1.600


### 12.5.2 Minibatches